In [2]:
import os
import glob
import cv2
import albumentations as A
import random
from collections import defaultdict
import shutil

class EmotionAugmenter:
    def __init__(self, dataset_multiplier=0.5):
        self.dataset_multiplier = dataset_multiplier  # 0.5 = add 50% more data
        self.class_names = ['anger', 'content', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

    def get_emotion_specific_augmentation(self, emotion_class):
        base = [A.HorizontalFlip(p=0.5), A.Rotate(limit=10, p=0.6)]

        if emotion_class in [0, 3, 6]:  # anger, fear, sad
            specific = [
                A.RandomGamma(gamma_limit=(80, 120), p=0.4),
                A.GaussNoise(var_limit=(5.0, 20.0), p=0.3),
            ]
        elif emotion_class in [1, 4, 5]:  # content, happy, neutral
            specific = [
                A.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.15, p=0.4),
                A.GaussianBlur(blur_limit=(1, 3), p=0.2),
            ]
        else:  # disgust, surprise
            specific = [
                A.ElasticTransform(alpha=1, sigma=30, alpha_affine=30, p=0.3),
                A.GridDistortion(num_steps=3, distort_limit=0.1, p=0.3),
            ]

        return A.Compose(base + specific, bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))
    
    def augment_single_image(self, img_path, label_path, output_dir, aug_idx):
        # 🔹 baca gambar dan konversi ke grayscale
        image = cv2.imread(img_path)
        if image is None or not os.path.exists(label_path):
            return 0
        image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.cvtColor(image_gray, cv2.COLOR_GRAY2BGR)  # tetap 3 channel untuk albumentations

        bboxes, labels = [], []
        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 5:
                    cls = int(float(parts[0]))
                    bboxes.append(list(map(float, parts[1:5])))
                    labels.append(cls)

        if not bboxes:
            return 0

        dominant_emotion = max(set(labels), key=labels.count)
        transform = self.get_emotion_specific_augmentation(dominant_emotion)

        os.makedirs(os.path.join(output_dir, 'images'), exist_ok=True)
        os.makedirs(os.path.join(output_dir, 'labels'), exist_ok=True)

        base_name = os.path.splitext(os.path.basename(img_path))[0]

        try:
            aug = transform(image=image, bboxes=bboxes, class_labels=labels)
            aug_img = aug['image']
            aug_boxes = aug['bboxes']
            aug_labels = aug['class_labels']

            if not aug_boxes:
                return 0

            img_out = os.path.join(output_dir, 'images', f"{base_name}_aug{aug_idx}.jpg")
            lbl_out = os.path.join(output_dir, 'labels', f"{base_name}_aug{aug_idx}.txt")

            cv2.imwrite(img_out, aug_img)
            with open(lbl_out, 'w') as f:
                for box, lbl in zip(aug_boxes, aug_labels):
                    f.write(f"{lbl} {' '.join(f'{x:.6f}' for x in box)}\n")
            return 1
        except:
            return 0

    def augment_dataset(self, input_dir, output_dir):
        images = glob.glob(os.path.join(input_dir, 'images', '*.jpg'))
        if not images:
            print("No images found")
            return
    
        # Group images by dominant class
        class_to_images = defaultdict(list)
        for img in images:
            base = os.path.splitext(os.path.basename(img))[0]
            lbl = os.path.join(input_dir, 'labels', f"{base}.txt")
            if not os.path.exists(lbl):
                continue
    
            labels = []
            with open(lbl, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) >= 5:
                        labels.append(int(float(parts[0])))
    
            if not labels:
                continue
    
            dominant = max(set(labels), key=labels.count)
            class_to_images[dominant].append((img, lbl))
    
        print("📊 Class distribution (original):")
        for cls, imgs in class_to_images.items():
            print(f"  {self.class_names[cls]}: {len(imgs)} images")
    
        oversample_factors = {0: 1.0, 5: 1.0, 6: 1.0}  # contoh untuk weak classes
    
        total_generated = 0
        for cls, img_list in class_to_images.items():
            multiplier = oversample_factors.get(cls, 0.5)
            n_to_aug = int(len(img_list) * multiplier)
            if n_to_aug == 0:
                print(f"Skipping augmentation for {self.class_names[cls]}")
                continue
    
            chosen = random.sample(img_list, min(len(img_list), n_to_aug))
            print(f"Augmenting {n_to_aug} images for class {self.class_names[cls]}")
    
            for idx, (img, lbl) in enumerate(chosen):
                total_generated += self.augment_single_image(img, lbl, output_dir, idx)
    
        print(f"✅ Done. Generated {total_generated} augmented samples.")
        
        # 🔹 copy data asli ke output_dir
        print("📂 Menyalin data asli ke folder augmented...")
        os.makedirs(os.path.join(output_dir, 'images'), exist_ok=True)
        os.makedirs(os.path.join(output_dir, 'labels'), exist_ok=True)

        for img in images:
            base = os.path.splitext(os.path.basename(img))[0]
            lbl = os.path.join(input_dir, 'labels', f"{base}.txt")

            out_img = os.path.join(output_dir, 'images', f"{base}.jpg")
            out_lbl = os.path.join(output_dir, 'labels', f"{base}.txt")

            shutil.copy(img, out_img)
            if os.path.exists(lbl):
                shutil.copy(lbl, out_lbl)

        print("✅ Data asli berhasil digabung dengan augmented dataset")


if __name__ == "__main__":
    augmenter = EmotionAugmenter()
    augmenter.augment_dataset(
        input_dir="/kaggle/working/Human-face-emotions-28/train",
        output_dir="/kaggle/working/augmented_emotion_train"
    )


📊 Class distribution (original):
  content: 788 images
  happy: 776 images
  fear: 838 images
  sad: 824 images
  disgust: 802 images
  neutral: 863 images
  surprise: 861 images
  anger: 834 images
Augmenting 394 images for class content
Augmenting 388 images for class happy
Augmenting 419 images for class fear


Argument(s) 'var_limit' are not valid for transform GaussNoise


Augmenting 824 images for class sad
Augmenting 401 images for class disgust


Argument(s) 'alpha_affine' are not valid for transform ElasticTransform


Augmenting 863 images for class neutral
Augmenting 430 images for class surprise
Augmenting 834 images for class anger
✅ Done. Generated 4553 augmented samples.
📂 Menyalin data asli ke folder augmented...
✅ Data asli berhasil digabung dengan augmented dataset


In [3]:
import os
import glob

# Path folder label
labels_path = "/kaggle/working/augmented_emotion_train/labels"
label_files = glob.glob(os.path.join(labels_path, "*.txt"))

print(f"Found {len(label_files)} label files")

fixed = 0
for file in label_files:
    new_lines = []
    changed = False
    with open(file, "r") as f:
        lines = f.readlines()
        for line in lines:
            parts = line.strip().split()
            if len(parts) >= 5:
                try:
                    # Konversi class id dari float ke int
                    cls_id = str(int(float(parts[0])))
                    rest = parts[1:]
                    new_line = " ".join([cls_id] + rest)
                    new_lines.append(new_line)
                    
                    # Kalau aslinya beda (misal "5.0" jadi "5"), tandai sudah diubah
                    if parts[0] != cls_id:
                        changed = True
                except ValueError:
                    print(f"⚠️  Error parsing line in {file}: {line.strip()}")
            else:
                # Simpan line apa adanya (misalnya kosong / invalid)
                new_lines.append(line.strip())
    
    # Tulis balik file kalau ada perubahan
    if changed:
        with open(file, "w") as f:
            f.write("\n".join(new_lines) + "\n")
        fixed += 1

print(f"✅ Cleaning selesai. {fixed} file sudah diperbaiki.")


Found 11139 label files
✅ Cleaning selesai. 4553 file sudah diperbaiki.


In [4]:
data_yaml_path = "/kaggle/working/augmented_emotion_train/data.yaml"

yaml_content = """path: /kaggle/working/augmented_emotion_train

train: images
val: /kaggle/working/Human-face-emotions-28/valid/images
test:  /kaggle/working/Human-face-emotions-28/test/images
nc: 8
names: ['anger','content','disgust','fear','happy','neutral','sad','surprise']
"""

with open(data_yaml_path, "w") as f:
    f.write(yaml_content)

print("✅ data.yaml written to", data_yaml_path)


✅ data.yaml written to /kaggle/working/augmented_emotion_train/data.yaml
